# Bayesian Classification for ECG Time-Series

> Copyright 2019 Dave Fernandes. All Rights Reserved.
> 
> Licensed under the Apache License, Version 2.0 (the "License");
> you may not use this file except in compliance with the License.
> You may obtain a copy of the License at
>
> http://www.apache.org/licenses/LICENSE-2.0
>  
> Unless required by applicable law or agreed to in writing, software
> distributed under the License is distributed on an "AS IS" BASIS,
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
> See the License for the specific language governing permissions and
> limitations under the License.

## Overview
This notebook classifies time-series for segmented heartbeats from ECG lead II recordings. Either of two models (CNN or RNN) can be selected from training and evaluation.
- Data for this analysis should be prepared using the `PreprocessECG.ipynb` notebook from this project.
- Original data is from: https://www.kaggle.com/shayanfazeli/heartbeat

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.layers as keras
import matplotlib.pyplot as plt
import pickle

tf.enable_eager_execution()

from google.colab import drive
drive.mount('/content/drive')

TRAIN_SET = '/content/drive/My Drive/Colab Notebooks/Data/train_set.pickle'
TEST_SET = '/content/drive/My Drive/Colab Notebooks/Data/test_set.pickle'

with open(TEST_SET, 'rb') as file:
    test_set = pickle.load(file)
    x_test = test_set['x']
    y_test = test_set['y']

with open(TRAIN_SET, 'rb') as file:
    train_set = pickle.load(file)
    x_train = train_set['x']
    y_train = train_set['y']

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Input functions for Estimator

In [0]:
def combined_dataset(features, labels):
    assert features.shape[0] == labels.shape[0]
    dataset = tf.data.Dataset.from_tensor_slices(({'time_series': features}, labels))
    return dataset

def class_for_element(features, labels):
    return labels

# For training
def train_input_fn():
    dataset = combined_dataset(x_train, y_train)
    return dataset.repeat().shuffle(500000).batch(200).prefetch(1)

# For evaluation and metrics
def eval_input_fn():
    dataset = combined_dataset(x_test, y_test)
    return dataset.batch(1000).prefetch(1)

### Define the model
#### Bayesian CNN Model
* The convolutional model is taken from: https://arxiv.org/pdf/1805.00794.pdf

Model consists of:
* An initial 1-D convolutional layer
* 5 repeated residual blocks (`same` padding)
* A fully-connected layer
* A linear layer with softmax output
* Flipout layers are used instead of standard layers

In [3]:
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/Models/BayesianCNN'

import tensorflow_probability as tfp

def conv_unit(unit, input_layer):
    s = '_' + str(unit)
    layer = tfp.layers.Convolution1DFlipout(name='Conv1' + s, filters=32, kernel_size=5, strides=1, padding='same', activation='relu')(input_layer)
    layer = tfp.layers.Convolution1DFlipout(name='Conv2' + s, filters=32, kernel_size=5, strides=1, padding='same', activation=None)(layer )
    layer = keras.Add(name='ResidualSum' + s)([layer, input_layer])
    layer = keras.Activation("relu", name='Act' + s)(layer)
    layer = keras.MaxPooling1D(name='MaxPool' + s, pool_size=5, strides=2)(layer)
    return layer

def cnn_model(input_shape, mode, params):
    time_series = tf.keras.layers.Input(shape=input_shape, dtype='float32')
    current_layer = tfp.layers.Convolution1DFlipout(filters=32, kernel_size=5, strides=1)(time_series)

    for i in range(3):
        current_layer = conv_unit(i + 1, current_layer)

    current_layer = keras.Flatten()(current_layer)
    current_layer = tfp.layers.DenseFlipout(32, name='FC1', activation='relu')(current_layer)
    logits = tfp.layers.DenseFlipout(5, name='Output')(current_layer)
    
    model = tf.keras.Model(inputs=time_series, outputs=logits, name='cnn_model')
    return model


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



### Estimator setup

In [4]:
import tensorflow.distributions as tfd

# Initial learning rate
INITIAL_LEARNING_RATE = 0.001

# Learning rate decay per LR_DECAY_STEPS steps (1.0 = no decay)
LR_DECAY_RATE = 0.5

# Number of steps for LR to decay by LR_DECAY_RATE
LR_DECAY_STEPS = 4000

# Threshold for gradient clipping
GRADIENT_NORM_THRESH = 10.0

# Select model to train
MODEL_FN = cnn_model

def classifier_fn(features, labels, mode, params):
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    input_layer = tf.feature_column.input_layer(features, params['feature_columns'])
    input_layer = tf.expand_dims(input_layer, -1)

    model = MODEL_FN(input_layer.shape[1:], mode, params)
    logits = model(input_layer)
    #logits = model
    labels_distribution = tfd.Categorical(logits=logits)

    # Compute the -ELBO as the loss, averaged over the batch size.
    neg_log_likelihood = -tf.reduce_mean(input_tensor=labels_distribution.log_prob(labels))
    kl = sum(model.losses) / 441000
    loss = neg_log_likelihood + kl
    
    # For prediction, exit here
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # For training and evaluation, compute the loss
    #neg_log_likelihood = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    accuracy = tf.metrics.accuracy(labels=labels, predictions=predicted_classes, name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

    # For training...
    global_step = tf.train.get_global_step()
    learning_rate = tf.train.exponential_decay(INITIAL_LEARNING_RATE, global_step, LR_DECAY_STEPS, LR_DECAY_RATE)

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    #optimizer = tf.contrib.estimator.clip_gradients_by_norm(optimizer, GRADIENT_NORM_THRESH)
    
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

feature_columns = [tf.feature_column.numeric_column('time_series', [187])]

regressor = tf.estimator.Estimator(
    model_fn=classifier_fn,
    model_dir=MODEL_DIR,
    params={
        'feature_columns': feature_columns,
    })

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/Colab Notebooks/Models/BayesianCNN', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f35c08f50f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Train model

In [0]:
regressor.train(train_input_fn, steps=4000)

info = regressor.evaluate(input_fn=eval_input_fn)

Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
In

### Compute metrics

In [0]:
import sklearn.metrics as skm

dataset_fn = eval_input_fn

predictions = regressor.predict(input_fn=dataset_fn)
y_predicted = []
y_prob = []

for i, value in enumerate(predictions):
    y_predicted.append(value['class_ids'])
    y_prob.append(value['probabilities'])
del predictions

y_test = np.reshape(y_test, (len(y_test), 1))

# Classification report
report = skm.classification_report(y_test, y_predicted)
print(report)

# Confusion matrix
conf = skm.confusion_matrix(y_test, y_predicted)
print(conf)

In [0]:
y_prob = np.array(y_prob)
correct = []
false_pos = []
false_neg = []

for k in range(5):
    for i in range(len(y_test)):
        if y_test[i] == k and y_predicted[i] == k:
            correct.append(y_prob[i, k])
        elif y_test[i] == k and y_predicted[i] != k:
            false_neg.append(y_prob[i, k])
        elif y_test[i] != k and y_predicted[i] == k:
            false_pos.append(y_prob[i, k])

n, bins, patches = plt.hist(correct, 20, (0, 1))
plt.xlabel('Probability')
plt.title('Correctly Classified')
plt.show();

n, bins, patches = plt.hist(false_pos, 20, (0, 1))
plt.xlabel('Probability')
plt.title('False Positives')
plt.show();

n, bins, patches = plt.hist(false_neg, 20, (0, 1))
plt.xlabel('Probability')
plt.title('False Negatives')
plt.show()